In [ ]:
import pandas as pd
import json
train_raw = pd.read_csv("./data/train.csv")
train_raw.head()

In [ ]:
train_proc = pd.concat([train_raw['prompt'].apply(json.loads), train_raw['response_a'].apply(json.loads), train_raw['response_b'].apply(json.loads), train_raw['winner_model_a'], train_raw['winner_model_b'], train_raw['winner_tie']], axis=1)
train_proc.head()

In [ ]:
print(train_proc['prompt'].apply(len).max(), train_proc['prompt'].apply(len).min())
print(train_proc['response_a'].apply(len).max(), train_proc['response_a'].apply(len).min())
print(train_proc['response_b'].apply(len).max(), train_proc['response_b'].apply(len).min())
all(train_proc['response_b'].apply(len)==train_proc['response_a'].apply(len)) and all(train_proc['response_b'].apply(len)==train_proc['prompt'].apply(len))

In [ ]:
train = train_proc.explode(['prompt', 'response_a', 'response_b'])
train.head()

In [ ]:
train['response_a'].isna().sum(), train['response_b'].isna().sum()

In [ ]:
train.fillna("",inplace=True)
train['response_a'].isna().sum(), train['response_b'].isna().sum()

In [ ]:
from transformers import AutoTokenizer, AutoModel

# an access token is needed if we use models like Mistral
def get_access_token() -> str:
    token = ""
    with open('token.txt', 'r') as f:
        token = f.read().strip()
    print(token)
    return token

In [ ]:
# tokenizer
text = "Replace me by any text you'd like."
# optimizations are possible by moving computation to GPU and using flash attention
# we just use cpu
tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')

train['combined_a'] = train.apply(lambda x: x['prompt'] + tokenizer.sep_token + x['response_a'], axis=1)
train['combined_b'] = train.apply(lambda x: x['prompt'] + tokenizer.sep_token + x['response_b'], axis=1)

In [ ]:
train.head()

In [ ]:
# __call__ will give ids
encoded_inputA = tokenizer(train['combined_a'].tolist(), return_tensors='pt', padding=True, truncation=True)
encoded_inputB = tokenizer(train['combined_b'].tolist(), return_tensors='pt', padding=True, truncation=True)

In [ ]:
print(encoded_inputA.input_ids.size(), len(train['combined_a'].tolist()))

In [ ]:
# models are stored in /home/<user>/.cache/huggingface
# find /home/<user> -type d -name huggingface
model = AutoModel.from_pretrained("distilbert/distilbert-base-uncased")

In [ ]:
def training(iterations: int, encoded_inputA, encoded_inputB) -> None:
    for i in range(iterations):
        outputA = model(encoded_inputA.input_ids, attention_mask=encoded_inputA.attention_mask)
        #outputB = model(encoded_inputB)
        print(outputA.size())

In [ ]:
model(encoded_inputA.input_ids, attention_mask=encoded_inputA.attention_mask)

In [ ]:
encoded_inputA.input_ids.size()